In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


In [2]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

Extract data from wikipedia in table

In [3]:
area_table = soup.find('table')
fields = area_table.find_all('td')

Create dataframe of postcode, borough and neighborhood

In [4]:
postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())

In [5]:
df_area = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_area.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_area.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Remove "Not assigned" values

In [6]:
df_area['Borough'].replace('Not assigned', np.nan, inplace=True)
df_area.dropna(subset=['Borough'], inplace=True)

In [7]:
df_area.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Group boroughs by postcode

In [8]:
df_area_code = df_area.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_area_code.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_area_code.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Query for any Borough that has a Neighbourhood value of "Not assigned"

In [9]:
df_area_code.query('Neighbourhood == "Not assigned"')

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


Update Neighbourhood to match name of Borough

In [10]:
df_area_code.loc[df_area_code.Neighbourhood == 'Not assigned', "Neighbourhood"] = df_area_code.Borough
df_area_code.query('Postcode == "M7A"')

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


Print Dataframe shape

In [17]:
df_area_code.shape

(103, 3)

In [61]:
df_area_code.to_csv("Toronto_Area_Code", index = False)

Get latitude and longitude coordinates for each postcode

In [68]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.columns = ['Postcode', 'Latitude', 'Longitude']

Merge lat and long coordinates with dataframe containing postcode, borough and neighbourhood

In [69]:
df_toronto = pd.merge(df_area_code, df_geo, on=['Postcode'], how='inner')
df_toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Print coordinates for Toronto, Ontario

In [70]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The City of Toronto's coordinates are {}, {}.".format(latitude, longitude))

The City of Toronto's coordinates are 43.653963, -79.387207.


Generate a map of Toronto neighbourhoods

In [71]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#FFF8DC',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Let's Explore Toronto

In [72]:
CLIENT_ID = 'QYMMUXQIM3DZQGCVJG3JKY5DXE4MXNT2CFH1BHTKM5BNN0XY' # your Foursquare ID
CLIENT_SECRET = 'YXDN0WTDQX4QQJ5FYB0H2AKHK2CQ5LNSGNI423QXIX3XELWS&' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QYMMUXQIM3DZQGCVJG3JKY5DXE4MXNT2CFH1BHTKM5BNN0XY
CLIENT_SECRET:YXDN0WTDQX4QQJ5FYB0H2AKHK2CQ5LNSGNI423QXIX3XELWS&


Select all neighbourhoods that are downtown

In [73]:
df_dt = df_toronto[df_toronto['Borough'].str.contains('Downtown')]

df_downtown = df_dt.reset_index(drop=True)
df_downtown

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


Create a map of the downtown neighbourhood

In [74]:
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighbourhood in zip(df_downtown['Latitude'], df_downtown['Longitude'],df_downtown['Borough'], df_downtown['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#FFF8DC',
        fill_opacity=0.5,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

Now it's time to explore one of the neighbourhoods. We will choose the last one listed in the dataframe

In [75]:
df_downtown.loc[17, 'Neighbourhood']

'Christie'

Now we will get latitude and longitude details for the neighbourhood Christie

In [76]:
neighbourhood_latitude = df_downtown.loc[17, 'Latitude'] 
neighbourhood_longitude = df_downtown.loc[17, 'Longitude'] 

neighbourhood_name = df_downtown.loc[17, 'Neighbourhood'] 

print('The neighbourhood of {} has a latitude {}, and longitude {}.'.format(neighbourhood_name, 
                                                                            neighbourhood_latitude, 
                                                                            neighbourhood_longitude))

The neighbourhood of Christie has a latitude 43.669542, and longitude -79.4225637.


What are the top 10 venues in a 500 metre radius in Christie

In [77]:
LIMIT = 10
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

In [81]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cdd4e01f594df57eb89b245'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Christie Pits',
  'headerFullLocation': 'Christie Pits, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 16,
  'suggestedBounds': {'ne': {'lat': 43.6740420045, 'lng': -79.41635411972038},
   'sw': {'lat': 43.6650419955, 'lng': -79.42877328027961}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4adcfd7cf964a5203e6321e3',
       'name': 'Fiesta Farms',
       'location': {'address': '200 Christie St',
        'crossStreet': 'at Essex St',
        'lat': 43.66847077052224,
        'lng': -79.42048512748114,
        'labeledLatLngs': [{'label': 'd

Use the function to get category details for venue

In [82]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [89]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Fiesta Farms,Grocery Store,43.668471,-79.420485
1,Contra Cafe,Café,43.669107,-79.426105
2,Starbucks,Coffee Shop,43.671585,-79.421366
3,Vinny’s Panini,Italian Restaurant,43.670679,-79.426148
4,Universal Grill,Diner,43.670550,-79.426541


In [88]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


Finally, let's create a map showing the Top 10 venues in the Christie neighborhood

In [98]:
map_top_ten = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, name, categories in zip(nearby_venues['lat'], nearby_venues['lng'],nearby_venues['name'], nearby_venues['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#FFF8DC',
        fill_opacity=0.5,
        parse_html=False).add_to(map_top_ten)  
    
map_top_ten